# Data camp models

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import seaborn as sns
#import plotly.express as px
import gc

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
weather_train = pd.read_csv('weather_train.csv')
weather_test = pd.read_csv('weather_test.csv')
building_metadata = pd.read_csv('building_metadata.csv')

In [3]:
train = train.merge(building_metadata, on='building_id', how='left')
alltrain = train.merge(weather_train, on=['site_id','timestamp'], how='left')
#del building_metadata,weather_train,train
gc.collect()

21

In [4]:
alltrain.timestamp = pd.to_datetime(alltrain['timestamp'])

In [5]:
varia = ['building_id','site_id', 'meter']

for col in varia:
    
    alltrain[col] =alltrain[col].astype('object')

In [6]:
alltrain['meter'] = pd.Categorical(alltrain['meter']).rename_categories({0: 'electricity', 
                                                                   1: 'chilledwater',
                                                                   2: 'steam', 
                                                                   3: 'hotwater'})

# FEATURE EXTRACTION

In [7]:
def rec_heur(x):
    
    if x in np.arange(6, 19):
        return 'journee'
    
    if x in np.arange(19, 23):
        return 'nuit'
    
    if x in [23, 0, 1, 2, 3, 4, 5]:
        return 'tard'

In [8]:
def discredit_var(x):
   
    
    if x <= 1951:
        return 'yearB_q1'
    
    if  1951 < x <= 1969:
        return 'yearB_q2'
    
    if 1969 < x <= 1993:
        return 'yearB_q3'
    
    if  1993 < x:
        return 'yearB_q4'

In [9]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [10]:
def preProcecing_df(df_):
    
    df=df_.copy()
    reduce_mem_usage(df)
    
    
    saison={3: 'printent',4:'printent',5:'printent',
          6: 'ete', 7: 'ete',8: 'ete', 
          9: 'automne', 10: 'automne', 11: 'automne', 
          1: 'hiver', 12: 'hiver', 2: 'hiver'}
      
    
    df['mois'] = df.timestamp.dt.month
    df['day'] = df.timestamp.dt.day
    df['heure'] = df.timestamp.dt.hour
    reduce_mem_usage(df)
    
    df['heureDiscredite'] = df['heure'].apply(rec_heur)
    reduce_mem_usage(df)
    
    df['week_end'] = [1 if x in [5,6] else 0 for x in df.day]
    reduce_mem_usage(df)
    df['saison'] = df['mois'].apply(lambda x: saison.get(x))
    reduce_mem_usage(df)
    median_group = df.groupby(['site_id'])['year_built'].transform('median')
    reduce_mem_usage(df)
    df['year_built'].fillna(median_group,inplace = True)
    reduce_mem_usage(df)
    df['year_built'].fillna(df['year_built'].median(), inplace=True)
    reduce_mem_usage(df)
    df['year_built'] = df['year_built'].apply(discredit_var) 
    reduce_mem_usage(df)
    df.floor_count.fillna(0,inplace = True)
    reduce_mem_usage(df)
    colonneAsNum=['air_temperature', 'dew_temperature','wind_direction']
    for col in colonneAsNum:
        median_group = df.groupby(['site_id', 'saison', 'week_end', 'primary_use'])[col].transform('median')
        df[col].fillna(median_group,inplace = True)
    reduce_mem_usage(df)                     
    for col in [ 'day', 'heure', 'timestamp', 
                "precip_depth_1_hr", "wind_speed", "sea_level_pressure", "cloud_coverage", "mois"]:
        del df[col]
    reduce_mem_usage(df)
    
    return df



In [11]:
X = preProcecing_df(alltrain)



Memory usage after optimization is: 1291.73 MB
Decreased by 48.1%
Memory usage after optimization is: 1349.57 MB
Decreased by 23.1%
Memory usage after optimization is: 1503.81 MB
Decreased by 0.0%
Memory usage after optimization is: 1523.09 MB
Decreased by 8.1%
Memory usage after optimization is: 1677.32 MB
Decreased by 0.0%
Memory usage after optimization is: 1677.32 MB
Decreased by 0.0%
Memory usage after optimization is: 1677.32 MB
Decreased by 0.0%
Memory usage after optimization is: 1677.32 MB
Decreased by 0.0%
Memory usage after optimization is: 1793.00 MB
Decreased by 0.0%
Memory usage after optimization is: 1793.00 MB
Decreased by 0.0%
Memory usage after optimization is: 1793.00 MB
Decreased by 0.0%
Memory usage after optimization is: 1426.69 MB
Decreased by 0.0%


In [12]:
X = reduce_mem_usage(X)

Memory usage after optimization is: 1426.69 MB
Decreased by 0.0%


In [13]:
X["building_id"] = X["building_id"].astype('category')
X["site_id"] = X["site_id"].astype('category')
X["primary_use"] = X["primary_use"].astype('category')
X["saison"] = X["saison"].astype('category')
X["heureDiscredite"] = X["heureDiscredite"].astype('category')

In [14]:
X.loc[  (X.site_id==0)&(X.meter == 'electricity') , 'meter_reading'] = X.loc[(X.site_id==0) & (X.meter == 'electricity') , 'meter_reading'] * 0.2931

In [15]:
X =  X[X['meter_reading']!= np.float(0)]
X['meter_reading']=np.log1p(X['meter_reading'])

In [16]:
X.shape # 14 colonnes

(18342124, 14)

In [17]:
def encodeur(df): 
    X_Encod=pd.concat([df, pd.get_dummies(df["primary_use"]) ], axis=1)
    reduce_mem_usage(X_Encod)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["saison"]) ], axis=1)
    reduce_mem_usage(X_Encod)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["heureDiscredite"]) ], axis=1)
    reduce_mem_usage(X_Encod)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["meter"]) ], axis=1)
    reduce_mem_usage(X_Encod)
    X_Encod=pd.concat([X_Encod, pd.get_dummies(df["year_built"]) ], axis=1)
    reduce_mem_usage(X_Encod)

    for col in ["primary_use",'year_built', "saison", "heureDiscredite", 'meter']:
        del X_Encod[col]


    return X_Encod

In [18]:
X_Encod = encodeur(X)

Memory usage after optimization is: 944.64 MB
Decreased by 3.6%
Memory usage after optimization is: 1014.61 MB
Decreased by 0.0%
Memory usage after optimization is: 1067.09 MB
Decreased by 0.0%
Memory usage after optimization is: 1137.06 MB
Decreased by 0.0%
Memory usage after optimization is: 1207.03 MB
Decreased by 0.0%


In [19]:
X_Encod.shape # 40 colonnes

(18342124, 40)

In [20]:
del X_Encod['building_id']

In [21]:
X_Encod.shape

(18342124, 39)

### Train Test split

In [22]:
from sklearn.model_selection import ShuffleSplit

def trainAndTest(DF):
    df= DF.copy()
    
    uniqueSite=list(pd.unique(df["site_id"]))
    rs = ShuffleSplit(n_splits=1, test_size=.3, random_state=0)
    for train_index, test_index in rs.split(uniqueSite):

        df['trainIndex'] = [1 if x in train_index else 0 for x in df.site_id]
        x_train = df[df['trainIndex']==1]
        y_train = x_train['meter_reading']

        x_test = df[df['trainIndex']==0]
        y_test = x_test['meter_reading']

    del x_train['trainIndex'] 
    del x_train['meter_reading'] 

    del x_test['trainIndex']
    del x_test['meter_reading']
    
    return x_train, y_train, x_test, y_test

In [23]:
 X_train, Y_train, X_test, Y_test = trainAndTest(X_Encod)

## MinMaxScaler

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [25]:
def minMax(DF, listColumns, scaler):
    df=DF.copy()
    for col in listColumns:
        df[col]=scaler.fit_transform(df[[col]])
        
    return df

In [26]:
listColumns= [ 'wind_direction',  'dew_temperature',
              'air_temperature', 'floor_count', 'square_feet']


X_train = minMax(X_train, listColumns, scaler)
X_test = minMax(X_test, listColumns, scaler)

In [27]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [28]:
X_train =reduce_mem_usage(X_train)
X_test =reduce_mem_usage(X_test)

Memory usage after optimization is: 576.20 MB
Decreased by 37.0%
Memory usage after optimization is: 315.92 MB
Decreased by 37.0%


In [29]:
X_train.to_csv ('x_train.csv', index = False, header=True)

In [30]:
X_test.to_csv ('x_test.csv', index = False, header=True)

In [31]:
Y_test.to_csv ('y_test.csv', index = False, header=True)

In [32]:
Y_train.to_csv ('y_train.csv', index = False, header=True)

## Models 

### Decision Tree

In [33]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(min_samples_split=200,min_samples_leaf=150)
dtr = dtr.fit(X_train,Y_train)

In [34]:
 from sklearn.metrics import mean_squared_error
    
y_pred_dtr = dtr.predict(X_test)
print(mean_squared_error(Y_test, y_pred_dtr)** 0.5)

1.70312156781


In [ ]:
from sklearn.ensemble import BaggingRegressor

br = BaggingRegressor(random_state=0)
br = br.fit(X_train,Y_train)

In [ ]:

y_pred_br = br.predict(X_test)
print(mean_squared_error(Y_test, y_pred_br)** 0.5)

## Soumission

In [35]:
test = test.merge(building_metadata, on='building_id', how='left')
alltest = test.merge(weather_test, on=['site_id','timestamp'], how='left')
#del building_metadata,weather_train,train
gc.collect()

21

In [36]:
alltest.timestamp = pd.to_datetime(alltest['timestamp'])

In [37]:
alltest['meter'] = pd.Categorical(alltest['meter']).rename_categories({0: 'electricity', 
                                                                   1: 'chilledwater',
                                                                   2: 'steam', 
                                                                   3: 'hotwater'})

In [38]:
data_test = preProcecing_df(alltest)

Memory usage after optimization is: 2147.36 MB
Decreased by 58.1%
Memory usage after optimization is: 2266.66 MB
Decreased by 26.9%
Memory usage after optimization is: 2584.79 MB
Decreased by 0.0%
Memory usage after optimization is: 2624.55 MB
Decreased by 9.6%
Memory usage after optimization is: 2942.68 MB
Decreased by 0.0%
Memory usage after optimization is: 2942.68 MB
Decreased by 0.0%
Memory usage after optimization is: 2942.68 MB
Decreased by 0.0%
Memory usage after optimization is: 2942.68 MB
Decreased by 0.0%
Memory usage after optimization is: 3181.27 MB
Decreased by 0.0%
Memory usage after optimization is: 3181.27 MB
Decreased by 0.0%
Memory usage after optimization is: 3181.27 MB
Decreased by 0.0%
Memory usage after optimization is: 2425.72 MB
Decreased by 0.0%


In [39]:
data_test["building_id"] = data_test["building_id"].astype('category')
data_test["site_id"] = data_test["site_id"].astype('category')
data_test["primary_use"] = data_test["primary_use"].astype('category')
data_test["saison"] = data_test["saison"].astype('category')
data_test["heureDiscredite"] = data_test["heureDiscredite"].astype('category')
data_test["year_built"] = data_test["year_built"].astype('category')

In [40]:
listColumns= [ 'wind_direction',  'dew_temperature',
              'air_temperature', 'floor_count', 'square_feet']

gc.collect()

77

In [41]:
X_final = encodeur(data_test)

Memory usage after optimization is: 1948.58 MB
Decreased by 0.0%
Memory usage after optimization is: 2107.65 MB
Decreased by 0.0%
Memory usage after optimization is: 2226.94 MB
Decreased by 0.0%
Memory usage after optimization is: 2386.01 MB
Decreased by 0.0%
Memory usage after optimization is: 2545.07 MB
Decreased by 0.0%


In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [43]:
def minMax(DF, listColumns, scaler):
    df=DF.copy()
    for col in listColumns:
        df[col]=scaler.fit_transform(df[[col]])
        
    return df

In [44]:
X_final = minMax(X_final, listColumns, scaler)
del data_test

In [45]:
X_final = X_final.drop(["row_id"],axis=1)

In [48]:
x_train = pd.read_csv('x_train.csv')

In [49]:
x_test = pd.read_csv('x_test.csv')

In [50]:
y_train = pd.read_csv('y_train.csv')

In [51]:
y_test = pd.read_csv('y_test.csv')

In [ ]:
dtr = DecisionTreeRegressor(min_samples_split=200,min_samples_leaf=150)
dtr = dtr.fit(x_train,y_train)


y_pred_final = dtr.predict(X_final)


In [ ]:
Ypred_finaldf = pd.DataFrame(data=y_pred_final)
Ypred_finaldf.to_csv('mypred.csv', index=False)

In [ ]:
submission  = pd.read_csv('sample_submission.csv')
submission['meter_reading'] = np.exp(y_pred_final)
submission.loc[submission['meter_reading']<0, 'meter_reading'] = 0
submission.to_csv('mysubmission.csv', index=False)

In [ ]:
submission

In [ ]:
br = BaggingRegressor(random_state=0)
br = br.fit(x_train,y_train)


y_pred_final2 = br.predict(X_final)

In [ ]:
Ypred_finaldf2 = pd.DataFrame(data=y_pred_final2)
Ypred_finaldf2.to_csv('mypred2.csv', index=False)

In [ ]:
submission2  = pd.read_csv('sample_submission.csv')
submission2['meter_reading'] = np.exp(y_pred_final2)
submission2.loc[submission2['meter_reading']<0, 'meter_reading'] = 0
submission2.to_csv('mysubmission2.csv', index=False)

In [ ]:
submission2